## This  is a notebook

In [6]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline


In [2]:
properties = pd.read_csv('../data/properties_2016.csv')
train_data = pd.read_csv('../data/train_2016_v2.csv')

C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Preprocessing

- merge data frames
- remove crappy features (missing > .75)
- make sure the types make sense


In [5]:
# Roel, doe je ding

## Feature engineering

In [ ]:
feature_engineering = 

## Model

In [ ]:
model = sklearn.N

## Pipeline

In [ ]:
pipe = Pipeline([('feature_engineering', feature_engineering),
                 ('model', model)
                 ]
)